## 05 PARK YEJUN

In [96]:
import pandas as pd
df = pd.read_csv('/Users/parkyejun/Downloads/5_SOCCER.csv')

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Columns: 106 entries, sofifa_id to rb
dtypes: float64(18), int64(44), object(44)
memory usage: 15.3+ MB


In [98]:
df.columns

Index(['sofifa_id', 'player_url', 'short_name', 'long_name', 'age', 'dob',
       'height_cm', 'weight_kg', 'nationality', 'club_name',
       ...
       'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb'],
      dtype='object', length=106)

In [99]:
# unique값에 대한 summary와 na값에 대한 summary가 츌력되는 함수
def create_summary(df):
    n_unique_df = {}
    n_na_df = {}
    for col in df.columns:
        n_na_df[col] = [df[col].isna().sum()/len(df)*100]
        n_unique_df[col] = [df[col].nunique()]
    n_unique_d = pd.DataFrame.from_dict(data= n_unique_df).T.sort_values(by =0).reset_index()
    n_na_d = pd.DataFrame(n_na_df).T.sort_values(by = 0).reset_index()
    n_na_d.columns = ['index', 'n_na']
    n_unique_d.columns = ['index', 'n_unique']
    return n_unique_d, n_na_d

unique_df , na_df = create_summary(df)

In [100]:
unique_df.head(20)

,index,n_unique
0,defending_marking,0
1,real_face,2
2,preferred_foot,2
3,league_rank,4
4,skill_moves,5
5,weak_foot,5
6,international_reputation,5
7,work_rate,9
8,contract_valid_until,9
9,nation_jersey_number,25


In [101]:
cat_list = unique_df.head(8).reset_index()['index'].tolist()
for col in cat_list:
    df[col] = df[col].astype('category')

In [102]:
na_df
rm_list = na_df[na_df['n_na']>5].reset_index()['index'].tolist()
for col in rm_list:
    df = df.drop(col, axis=1)

In [103]:
rm_list

['joined',
 'release_clause_eur',
 'physic',
 'defending',
 'dribbling',
 'passing',
 'shooting',
 'pace',
 'player_traits',
 'gk_handling',
 'gk_kicking',
 'gk_reflexes',
 'gk_speed',
 'gk_positioning',
 'gk_diving',
 'player_tags',
 'nation_position',
 'nation_jersey_number',
 'loaned_from',
 'defending_marking']

In [104]:
lst_unnecessary  = [
    'short_name',
    'long_name',
    'dob',
    'nationality',
    'club_name',
    'league_name',
    'team_position',
    'team_jersey_number',
    'contract_valid_until',
    'real_face',
    'ls',
    'st',
    'rs',
    'lw',
    'lf',
    'cf',
    'rf',
    'rw',
    'lam',
    'cam',
    'ram',
    'lm',
    'lcm',
    'cm',
    'rcm',
    'rm',
    'lwb',
    'ldm',
    'cdm',
    'rdm',
    'rwb',
    'lb',
    'lcb',
    'cb',
    'rcb',
    'rb',
]



In [105]:
#wage_eur은 데이터 누수이므로 제거
df.drop(['sofifa_id', 'player_url']+lst_unnecessary+['wage_eur'], axis=1, inplace=True)

In [108]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import pandas as pd
# IQR로 outlier 처리
def check_outlier(df, threshold=1.5):
        for col in df.select_dtypes(include=['int', 'float']):
            if col != 'value_eur':
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - threshold * IQR
                upper_bound = Q3 + threshold * IQR
                df[col] = df[col].clip(lower_bound, upper_bound)
        return df

#모든 전처리를 한번에 해주는 함수
def d_converter(df, unique_threshold = 5):
    df = df.copy()
    
    # 이상치 처리
    df = check_outlier(df, threshold=1.5)

    # numeric 변수 minmax scaling
    for col in df.select_dtypes(include=['int', 'float']):
        if col != 'value_eur':
        #평균으로 결측치 처리... 다른 방법으로 할 수 있으면 해보십숑 interpolate도 가능
            df[col] = df[col].fillna(df[col].mean())
            scaler = MinMaxScaler()
            df[col] = scaler.fit_transform(df[[col]])

    # categorical 변수를 인코딩
    for col in df.select_dtypes(include=['object', 'category']):
        if col != 'value_eur':
            df[col] =df[col].fillna(df[col].mode())
            #5개 이상이면...label encoding인데 기준 바꾸어도됨 상황따라서
            if df[col].nunique() >= unique_threshold:
                le = LabelEncoder()
                df[col] = le.fit_transform(df[col])
            else:
                dummies = pd.get_dummies(df[col], prefix=col)
                df = pd.concat([df.drop(columns=[col]), dummies], axis=1)
                for new_col in dummies.columns:
                    if new_col not in df.columns:
                        df[new_col] = 0
                    df[new_col] = df[new_col].astype('int')
    return df


In [111]:
df = d_converter(df)
df.rename(columns={'value_eur':'Value'}, inplace=True)